In [1]:
import pymysql, os
import numpy as np
from astropy import units as u
from astropy.coordinates import SkyCoord
import pandas, alminer, glob
import matplotlib.pyplot as plt

In [2]:
from astropy.cosmology import FlatLambdaCDM
cosmo = FlatLambdaCDM(H0=70, Om0=0.3)

In [3]:
from skaha.session import Session
import nest_asyncio
nest_asyncio.apply()

# Read in sample to be reduced

In [9]:
fpath = '/arc/projects/salvage/ALMA_reduction/samples/'
file  =  'almaquest_sample.txt'

objID_sample, year_sample, name_sample, muid_sample, guid_sample, auid_sample, proj_sample = np.loadtxt(fpath+file, unpack = True, dtype = str, usecols = [0,11,12,13,14,15,16])
z_sample, mass_sample, rpetro_sample, ra_sample, dec_sample, res_sample, mrs_sample, AL_sample, AC_sample, TP_sample = np.loadtxt(fpath+file, unpack = True, dtype = float, usecols = [1,2,3,4,5,6,7,8,9,10])

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


# ALMiner Download and Unzip with headless jobs

Here we iterate over the **ALMA Source Name**s that were matched to SDSS targets within 10" **and** cover the appropriate frequency range given the matched SDSS target's spectroscopic redshift. For each ALMA Source Name we download **ALL** observations with that specific source name, regardless of RA/Dec position. Bear in mind that for some low-z ALMA sources, there may be observations that get downloaded that have an RA and Dec that better match a higher redshift SDSS target. *That* target is not connected to this observation.

The 10" and redshift coverage check is performed in the previous stage (salvage-sample-selection.ipynb). All names loaded from the file in the previous cell should be vetted for coverage). 

**IF** there are multiple unique ALMA Source Names in the ALMA science archive that match to an SDSS target and have an appropriate frequency range, that SDSS target will be downloaded twice, but included in the same directory (indexed by its SDSS objID). Later when making the PHANGS-ALMA pipeline Measurement Set Key, these individual observations will be under the **SAME target name** and therefore should be imaged together.

In [57]:
# select appropriate resources

image = "images.canfar.net/skaha/astroml:latest"
cmd = '/arc/projects/salvage/ALMA_reduction/bash_scripts/download_and_unzip.sh'
ram=4 ### testing with less RAM this time!!!
cores=2

session_ids = []

rerun_targets = ['587731513694355564']

# loop over galaxies and launch jobs
for i in range(len(objID_sample)):

    ID = objID_sample[i]
    RA = ra_sample[i]
    DEC = dec_sample[i]
    NAME = name_sample[i]
    MUID = muid_sample[i]
    GUID = guid_sample[i]
    AUID = auid_sample[i]
    PROJ = proj_sample[i]

    # running only on problem galaxies
    #if ID not in rerun_targets:
    #    continue

    TAR1 = PROJ + '_uid___' + MUID.split('/')[2] + '_' + MUID.split('/')[3] + '_' + MUID.split('/')[4] + '_001_of_001.tar'
    TAR2 = PROJ + '_uid___' + MUID.split('/')[2] + '_' + MUID.split('/')[3] + '_' + MUID.split('/')[4] + '_auxiliary.tar'
    ASDM = PROJ + '_uid___' + AUID.split('/')[2] + '_' + AUID.split('/')[3] + '_' + AUID.split('/')[4] + '.asdm.sdm.tar'
    #ASDM = PROJ + '_uid___' + AUID.split('/')[2] + '_' + AUID.split('/')[3] + '_*.asdm.sdm.tar'

    # launch headless session on the science platform
    session = Session()
    session_id = session.create(
        name  = ID,
        image = image,
        cores = cores,
        ram   = ram,
        kind  = "headless",
        cmd   = cmd,
        args  = f'{ID} {NAME}'
    )

    print("Sesion ID: {}".format(session_id[0]))
    session_ids.append(session_id[0])


2024-03-20 20:01:54,874 - skaha-client-skaha.session - INFO - Creating 1 session(s) with parameters:
2024-03-20 20:01:54,874 INFO Creating 1 session(s) with parameters:
2024-03-20 20:01:54,876 - skaha-client-skaha.session - INFO - {'name': '587724198283247682', 'image': 'images.canfar.net/skaha/astroml:latest', 'cores': 2, 'ram': 4, 'kind': 'headless', 'cmd': '/arc/projects/salvage/ALMA_reduction/bash_scripts/download_and_unzip.sh', 'args': '587724198283247682 CP_9195_9101 2019.1.00260.S_uid___A001_X1465_X3529_001_of_001.tar 2019.1.00260.S_uid___A001_X1465_X3529_auxiliary.tar 2019.1.00260.S_uid___A002_Xe37224_Xd650.asdm.sdm.tar', 'env': {}}
2024-03-20 20:01:54,876 INFO {'name': '587724198283247682', 'image': 'images.canfar.net/skaha/astroml:latest', 'cores': 2, 'ram': 4, 'kind': 'headless', 'cmd': '/arc/projects/salvage/ALMA_reduction/bash_scripts/download_and_unzip.sh', 'args': '587724198283247682 CP_9195_9101 2019.1.00260.S_uid___A001_X1465_X3529_001_of_001.tar 2019.1.00260.S_uid___A

Sesion ID: g3rbr0zi


2024-03-20 20:01:57,818 - skaha-client-skaha.session - INFO - Creating 1 session(s) with parameters:
2024-03-20 20:01:57,818 INFO Creating 1 session(s) with parameters:
2024-03-20 20:01:57,819 - skaha-client-skaha.session - INFO - {'name': '587730848501530706', 'image': 'images.canfar.net/skaha/astroml:latest', 'cores': 2, 'ram': 4, 'kind': 'headless', 'cmd': '/arc/projects/salvage/ALMA_reduction/bash_scripts/download_and_unzip.sh', 'args': '587730848501530706 PM1_8615_1901 2019.1.00260.S_uid___A001_X1465_X34e5_001_of_001.tar 2019.1.00260.S_uid___A001_X1465_X34e5_auxiliary.tar 2019.1.00260.S_uid___A002_Xe37224_X110df.asdm.sdm.tar', 'env': {}}
2024-03-20 20:01:57,819 INFO {'name': '587730848501530706', 'image': 'images.canfar.net/skaha/astroml:latest', 'cores': 2, 'ram': 4, 'kind': 'headless', 'cmd': '/arc/projects/salvage/ALMA_reduction/bash_scripts/download_and_unzip.sh', 'args': '587730848501530706 PM1_8615_1901 2019.1.00260.S_uid___A001_X1465_X34e5_001_of_001.tar 2019.1.00260.S_uid_

Sesion ID: awvpzmed


2024-03-20 20:02:01,002 - skaha-client-skaha.session - INFO - Creating 1 session(s) with parameters:
2024-03-20 20:02:01,002 INFO Creating 1 session(s) with parameters:
2024-03-20 20:02:01,005 - skaha-client-skaha.session - INFO - {'name': '587731173842027508', 'image': 'images.canfar.net/skaha/astroml:latest', 'cores': 2, 'ram': 4, 'kind': 'headless', 'cmd': '/arc/projects/salvage/ALMA_reduction/bash_scripts/download_and_unzip.sh', 'args': '587731173842027508 587731173842027508_8623-6104 2018.1.00541.S_uid___A001_X133d_Xea3_001_of_001.tar 2018.1.00541.S_uid___A001_X133d_Xea3_auxiliary.tar 2018.1.00541.S_uid___A002_Xd90607_X46d0.asdm.sdm.tar', 'env': {}}
2024-03-20 20:02:01,005 INFO {'name': '587731173842027508', 'image': 'images.canfar.net/skaha/astroml:latest', 'cores': 2, 'ram': 4, 'kind': 'headless', 'cmd': '/arc/projects/salvage/ALMA_reduction/bash_scripts/download_and_unzip.sh', 'args': '587731173842027508 587731173842027508_8623-6104 2018.1.00541.S_uid___A001_X133d_Xea3_001_of_0

...

Sesion ID: t25fv6dp


2024-03-20 20:03:41,127 - skaha-client-skaha.session - INFO - Creating 1 session(s) with parameters:
2024-03-20 20:03:41,127 INFO Creating 1 session(s) with parameters:
2024-03-20 20:03:41,128 - skaha-client-skaha.session - INFO - {'name': '587724234256351256', 'image': 'images.canfar.net/skaha/astroml:latest', 'cores': 2, 'ram': 4, 'kind': 'headless', 'cmd': '/arc/projects/salvage/ALMA_reduction/bash_scripts/download_and_unzip.sh', 'args': '587724234256351256 PM1_9195_3703 2019.1.00260.S_uid___A001_X1465_X351d_001_of_001.tar 2019.1.00260.S_uid___A001_X1465_X351d_auxiliary.tar 2019.1.00260.S_uid___A002_Xe2eefc_X2679.asdm.sdm.tar', 'env': {}}
2024-03-20 20:03:41,128 INFO {'name': '587724234256351256', 'image': 'images.canfar.net/skaha/astroml:latest', 'cores': 2, 'ram': 4, 'kind': 'headless', 'cmd': '/arc/projects/salvage/ALMA_reduction/bash_scripts/download_and_unzip.sh', 'args': '587724234256351256 PM1_9195_3703 2019.1.00260.S_uid___A001_X1465_X351d_001_of_001.tar 2019.1.00260.S_uid__

Sesion ID: v6ps2m5a


2024-03-20 20:03:43,609 - skaha-client-skaha.session - INFO - Creating 1 session(s) with parameters:
2024-03-20 20:03:43,609 INFO Creating 1 session(s) with parameters:
2024-03-20 20:03:43,610 - skaha-client-skaha.session - INFO - {'name': '587739719757922345', 'image': 'images.canfar.net/skaha/astroml:latest', 'cores': 2, 'ram': 4, 'kind': 'headless', 'cmd': '/arc/projects/salvage/ALMA_reduction/bash_scripts/download_and_unzip.sh', 'args': '587739719757922345 8952-6104 2017.1.01093.S_uid___A001_X1288_Xf15_001_of_001.tar 2017.1.01093.S_uid___A001_X1288_Xf15_auxiliary.tar 2017.1.01093.S_uid___A002_Xcf04b6_Xf64.asdm.sdm.tar', 'env': {}}
2024-03-20 20:03:43,610 INFO {'name': '587739719757922345', 'image': 'images.canfar.net/skaha/astroml:latest', 'cores': 2, 'ram': 4, 'kind': 'headless', 'cmd': '/arc/projects/salvage/ALMA_reduction/bash_scripts/download_and_unzip.sh', 'args': '587739719757922345 8952-6104 2017.1.01093.S_uid___A001_X1288_Xf15_001_of_001.tar 2017.1.01093.S_uid___A001_X1288_

Sesion ID: c0eiwwgb
Sesion ID: exasiaq1


In [58]:
for i,sid in enumerate(session_ids):

    info = session.info(sid)[0]
    print(info['id'], info['status'])

g3rbr0zi Running
awvpzmed Succeeded
iq00gjlr Running
f0xxqwg8 Running
m8n45akz Succeeded
i9jba76d Succeeded
jnek4ngo Succeeded
d21ont4l Succeeded
q97e5ahi Succeeded
vqmhrslo Succeeded
mwbs3s7f Succeeded
p5yqplg6 Succeeded
rnsaso99 Running
a4d90it7 Running
jg530l2n Running
yx9ueezz Running
a11nxzsf Running
aeqzz8tc Succeeded
gyssnc8y Succeeded
hpa9rdck Running
rzjdc3d3 Running
voptz4si Running
ak58418i Running
d340f97m Running
u2dhf1xt Running
nwq83721 Running
mcuweqa6 Running
jkplb8a0 Running
hhclloo3 Running
w2r3zbvh Pending
eabv2nq6 Pending
l5ad0iob Succeeded
bggl9n60 Running
j7ro3qi1 Pending
p7mtpnhi Running
gljdxk96 Running
x0ojhnvg Running
nnvkpap1 Running
svw5m4sr Running
vfqgknew Succeeded
y8ppht4f Running
ewyold59 Running
cohflhjs Running
rray6k17 Running
r2qq9arn Running
b6azeq70 Succeeded
h1c12uff Running
nkhsxtzb Running
qat2joyn Running
rw0yip5h Pending
mdo6uww5 Pending
b953tr4d Pending
lqzwtb9e Pending
t25fv6dp Pending
v6ps2m5a Succeeded
c0eiwwgb Running
exasiaq1 Succeeded

In [22]:
for sid in session_ids:

    print(session.logs(sid)[sid])

## Run ScriptForPI.py

In [62]:
# select appropriate resources
image = "images.canfar.net/casa-5/casa:5.6.1-8-pipeline"
#image = "images.canfar.net/casa-4/casa:4.2.2-pipe"
cmd = '/arc/projects/salvage/ALMA_reduction/bash_scripts/restore_calibration.sh'
ram=8
cores=2

session_ids = []

#rerun_targets = ['587731513153159199', '587727220865040712']
rerun_targets = ['587731513694355564']


# loop over galaxies and launch jobs
for i in range(len(objID_sample)):

    ID = objID_sample[i]
    MUID = muid_sample[i]
    PROJ = proj_sample[i]
    YEAR = int(year_sample[i])

    UID =  MUID.split('/')[2] + '_' + MUID.split('/')[3] + '_' + MUID.split('/')[4]

    # select the appropriate CASA version following https://almascience.nrao.edu/processing/science-pipeline
    if YEAR < 2018:
        image = "images.canfar.net/casa-4/casa:4.2.2-pipe"
    elif YEAR < 2021:
        image = "images.canfar.net/casa-5/casa:5.6.1-8-pipeline"
    else:
        image = "images.canfar.net/casa-6/casa:6.5.6-22"

    # running only on problem galaxies
    if ID not in rerun_targets:
        continue

    # search all directories in the ALMA project folder for the relevant data
    for root, dirs, files in os.walk(f"/arc/projects/salvage/ALMA_data/{ID}/{PROJ}/"):
        if "member.uid___"+UID in dirs:
            PATH = os.path.join(root, "member.uid___"+UID)

    # launch headless session on the science platform
    session = Session()
    session_id = session.create(
        name  = ID,
        image = image,
        cores = cores,
        ram   = ram,
        kind  = "headless",
        cmd   = cmd,
        args  = f'{ID} {PROJ} {PATH}'
    )

    print("Sesion ID: {}".format(session_id[0]))
    session_ids.append(session_id[0])


2024-03-20 23:10:42,629 - skaha-client-skaha.session - INFO - Creating 1 session(s) with parameters:
2024-03-20 23:10:42,629 INFO Creating 1 session(s) with parameters:
2024-03-20 23:10:42,632 - skaha-client-skaha.session - INFO - {'name': '587731513694355564', 'image': 'images.canfar.net/casa-5/casa:5.6.1-8-pipeline', 'cores': 2, 'ram': 8, 'kind': 'headless', 'cmd': '/arc/projects/salvage/ALMA_reduction/bash_scripts/restore_calibration.sh', 'args': '587731513694355564 2018.1.00541.S /arc/projects/salvage/ALMA_data/587731513694355564/2018.1.00541.S/science_goal.uid___A001_X133d_Xe79/group.uid___A001_X133d_Xe7a/member.uid___A001_X133d_Xe7b', 'env': {}}
2024-03-20 23:10:42,632 INFO {'name': '587731513694355564', 'image': 'images.canfar.net/casa-5/casa:5.6.1-8-pipeline', 'cores': 2, 'ram': 8, 'kind': 'headless', 'cmd': '/arc/projects/salvage/ALMA_reduction/bash_scripts/restore_calibration.sh', 'args': '587731513694355564 2018.1.00541.S /arc/projects/salvage/ALMA_data/587731513694355564/20

Sesion ID: fihyych9


In [314]:
print(session.info(session_id))

[{'id': 'bc0qf1d5', 'userid': 'swilkinson', 'appid': '<none>', 'image': 'images.canfar.net/casa-5/casa:5.6.1-8-pipeline', 'type': 'headless', 'status': 'Running', 'name': '587726032772333771-1', 'startTime': '2024-02-13T08:35:50Z', 'expiryTime': '2024-02-27T08:35:50Z', 'connectURL': 'not-applicable', 'requestedRAM': '16G', 'requestedCPUCores': '2', 'requestedGPUCores': '0', 'ramInUse': '636M', 'gpuRAMInUse': '<none>', 'cpuCoresInUse': '0.687', 'gpuUtilization': '<none>'}]


In [65]:
for i,sid in enumerate(session_ids):

    try:
        info = session.info(sid)[0]
        print(info['name'], info['id'], info['status'])

    except:

        print(session.info(sid))

587731513694355564-1 fihyych9 Running


In [61]:
for sid in session_ids:
    
    print(session.logs(sid)[sid])

exec /arc/projects/salvage/ALMA_reduction/bash_scripts/restore_calibration.sh: exec format error



## Make Measurement Set Key

In [67]:
key_dir = '/arc/projects/salvage/phangs_imaging_scripts/phangs-alma_keys/'

ms_key_out = 'ms_file_key_salvage.txt'
ms_key_tmp = 'ms_file_key_template.txt'

# prep by copy current ms key to a template file with "__DATA__" where data should go (DONE)

# get template table (formatting with no file paths)
out = open(key_dir + ms_key_tmp, 'r')
out_data = out.read()
out.close()

# wipe old content from output file
out = open(key_dir + ms_key_out, 'w')
out.close()

# initialize objID input tracker
objID_in_file = []

# initialize string
out_str = ''

for i in range(len(objID_sample)):

    ID = objID_sample[i]
    MUID = muid_sample[i]
    PROJ = proj_sample[i]

    UID =  MUID.split('/')[2] + '_' + MUID.split('/')[3] + '_' + MUID.split('/')[4]

    # search all directories in the ALMA project folder for the relevant data
    for root, dirs, files in os.walk(f"/arc/projects/salvage/ALMA_data/{ID}/{PROJ}/"):
        if "member.uid___"+UID in dirs:
            PATH = os.path.join(root, "member.uid___"+UID)

    # prep file path for PHANGS-ALMA pipeline
    ms_root = '/arc/projects/salvage/ALMA_data/'
    ms_filepath = PATH.replace(ms_root, '') # wipe ms_root directory so it can be added separately
    ms_filepath += '/calibrated/' # add calibrated on the end so that it points to the calibrated data

    # until I discover more nuance, take all measurement set files from this directory and let PHANGS deal with them
    ms_files = glob.glob(ms_root + ms_filepath + '*.ms*')

    print(ms_root + ms_filepath, os.path.isdir(ms_root + ms_filepath), len(ms_files))

    if len(ms_files)<1:
        #checking if files are missing because of bug in previous stages or this one...
        print('No MS files found in ', ms_filepath)

    # check if there are visibilities for this target from another project and add on top if there is...
    init = len(np.array(objID_in_file)[np.array(objID_in_file)==ID])

    # add each measurement set to ms key
    for j in range(init, init + len(ms_files)):

        ms_path = ms_files[j-init].replace(ms_root, '') # wipe ms_root directory so it can be added separately
        obs_num = j+1 # record different measurement sets as different observations
    
        out_str += f"{ID} {ID}_{obs_num} all 12m {obs_num} {ms_path}\n" 
        # need to accomodate other observations of the same objID?

        objID_in_file.append(ID)

# replace placeholder "__DATA__" in template with formatted data
out_data = out_data.replace('__DATA__', out_str)
    
# write to new key file
out = open(key_dir + ms_key_out, 'w')
out.write(out_data)
out.close()

/arc/projects/salvage/ALMA_data/587724198283247682/2019.1.00260.S/science_goal.uid___A001_X1465_X3527/group.uid___A001_X1465_X3528/member.uid___A001_X1465_X3529/calibrated/ True 2
/arc/projects/salvage/ALMA_data/587730848501530706/2019.1.00260.S/science_goal.uid___A001_X1465_X34e3/group.uid___A001_X1465_X34e4/member.uid___A001_X1465_X34e5/calibrated/ True 1
/arc/projects/salvage/ALMA_data/587730847423791750/2019.1.00260.S/science_goal.uid___A001_X1465_X3513/group.uid___A001_X1465_X3514/member.uid___A001_X1465_X3515/calibrated/ True 1
/arc/projects/salvage/ALMA_data/587731173842027508/2018.1.00541.S/science_goal.uid___A001_X133d_Xea1/group.uid___A001_X133d_Xea2/member.uid___A001_X133d_Xea3/calibrated/ True 2
/arc/projects/salvage/ALMA_data/587730847427265072/2017.1.01093.S/science_goal.uid___A001_X1288_Xeef/group.uid___A001_X1288_Xef0/member.uid___A001_X1288_Xef1/calibrated/ True 1
/arc/projects/salvage/ALMA_data/587731173842027508/2018.1.00558.S/science_goal.uid___A001_X133d_Xf47/group

## Make distance key

In [68]:
key_dir = '/arc/projects/salvage/phangs_imaging_scripts/phangs-alma_keys/'

dist_key_out = 'distance_key_salvage.txt'
dist_key_tmp = 'distance_key_template.txt'

# prep by copy current distance key to a template file with "__DATA__" where data should go (DONE)

# get template table (formatting with no coordinates)
out = open(key_dir + dist_key_tmp, 'r')
out_data = out.read()
out.close()

# initialize string
out_str = ''

for i in range(len(objID_sample)):

    ID   = objID_sample[i]
    NAME = name_sample[i]
    Z    = z_sample[i]

    DIST = cosmo.angular_diameter_distance(Z).value

    out_str += f"{ID},{DIST}\n" 
    # need to accomodate other observations of the same objID?

# replace __DATA__ in template with formatted data
out_data = out_data.replace('__DATA__', out_str)

# write to new key file
out = open(key_dir + dist_key_out, 'w')
out.write(out_data)
out.close()

## Make target definitions key

In [69]:
key_dir = '/arc/projects/salvage/phangs_imaging_scripts/phangs-alma_keys/'

targ_key_out = 'target_definitions_salvage.txt'
targ_key_tmp = 'target_definitions_template.txt'

# prep by copy current target definitions key to a template file with "__DATA__" where data should go (DONE)

# prep for sys vel calculation
restfreq = 115.27120 * u.GHz  # rest frequency of 12 CO 1-0 in GHz
freq_to_vel = u.doppler_radio(restfreq)

# get template table (formatting with no coordinates)
out = open(key_dir + targ_key_tmp, 'r')
out_data = out.read()
out.close()

# initialize string
out_str = ''

# format the ra and dec's for html table
for i in range(len(objID_sample)):

    ID   = objID_sample[i]
    NAME = name_sample[i]
    Z    = z_sample[i]
    RA   = ra_sample[i]
    DEC  = dec_sample[i]

    c = SkyCoord(ra= RA*u.degree, dec= DEC*u.degree, frame='icrs')
    coord_str = c.to_string('hmsdms')

    sys_vel = (restfreq/(1+Z)).to(u.km / u.s, equivalencies=freq_to_vel).value
    velwidth = 1000

    out_str += f"{ID} {coord_str} {sys_vel} {velwidth}\n" 
    # need to accomodate other observations of the same objID?

# replace __DATA__ in template with formatted data
out_data = out_data.replace('__DATA__', out_str)

# write to new key file
out = open(key_dir + targ_key_out, 'w')
out.write(out_data)
out.close()

## Run PHANGS Pipeline

In [70]:
# select appropriate resources

#image = "images.canfar.net/skaha/astroml:latest"
image = "images.canfar.net/casa-6/casa:6.5.6-22"
cmd = '/arc/projects/salvage/ALMA_reduction/bash_scripts/run_PHANGS_pipeline.sh'
ram=8
cores=2

session_ids = []

#rerun_targets = ['587724198283247682','587731512085708963','587727220865040712',\
#'587731511545233578','587731513694355564','587731513153159199']

#rerun_targets = [ '587727220865040712']
rerun_targets = ['587731513694355564']

# loop over galaxies and launch jobs
for i in range(len(objID_sample)):

    ID = objID_sample[i]
    NAME = name_sample[i]

    # running only on problem galaxies
    if ID not in rerun_targets:
        continue
    
    # launch headless session on the science platform
    session = Session()
    session_id = session.create(
        name  = ID,
        image = image,
        cores = cores,
        ram   = ram,
        kind  = "headless",
        cmd   = cmd,
        args  = f'{NAME} {ID}'
    )

    print("Sesion ID: {}".format(session_id[0]))
    session_ids.append(session_id[0])


2024-03-21 04:36:31,130 - skaha-client-skaha.session - INFO - Creating 1 session(s) with parameters:
2024-03-21 04:36:31,130 INFO Creating 1 session(s) with parameters:
2024-03-21 04:36:31,131 - skaha-client-skaha.session - INFO - {'name': '587731513694355564', 'image': 'images.canfar.net/casa-6/casa:6.5.6-22', 'cores': 2, 'ram': 8, 'kind': 'headless', 'cmd': '/arc/projects/salvage/ALMA_reduction/bash_scripts/run_PHANGS_pipeline.sh', 'args': '587731513694355564_8155-6102 587731513694355564', 'env': {}}
2024-03-21 04:36:31,131 INFO {'name': '587731513694355564', 'image': 'images.canfar.net/casa-6/casa:6.5.6-22', 'cores': 2, 'ram': 8, 'kind': 'headless', 'cmd': '/arc/projects/salvage/ALMA_reduction/bash_scripts/run_PHANGS_pipeline.sh', 'args': '587731513694355564_8155-6102 587731513694355564', 'env': {}}
2024-03-21 04:36:35,709 - skaha-client-skaha.session - ERROR - 400 Client Error: Bad Request for url: https://ws-uv.canfar.net/skaha/v0/session?name=587731513694355564-1&image=images.can

IndexError: list index out of range

In [107]:
# loop over galaxies and launch jobs
for i in range(len(objID_sample[0:10])):

    ID = objID_sample[0:10][i]
    NAME = name_sample[0:10][i]

    print(NAME, ID)

587731173842027508_8623-6104 587731173842027508
8623-6104 587731173842027508
8655-3701 587731185665638418
COSMOS-1411106 587726033309335775
COSMOS-3666 587726032772333771
COSMOS-20062 587727944033960150
587731511545626673_8081-3704 587731511545626673
587731187797393472_8615-3703 587731187797393472
8077-9101 587731511542022280
zwicky_3146 588010359075700997


In [16]:
for i,sid in enumerate(session_ids):

    info = session.info(sid)[0]
    print(info['id'], info['status'])

es64pzz0 Pending
so5romty Pending
bfss4e1s Pending
owhz9cyo Pending
pqouxibu Pending
lu64x1ka Pending
jj9051do Pending
gt5epqcc Pending
hborild1 Pending
qdwzzaqx Pending
hk3v6yv7 Pending
sq69gvef Pending
aza7ylwd Pending
h6jqwyhf Pending
dy4afl17 Pending
ilk8xxiy Pending
lgsb64dt Pending
x3xf2law Pending
ot0t9n77 Pending
rq2szm8b Succeeded
ys3bl133 Pending
ieuy7at1 Pending


2024-03-18 00:06:46,174 - skaha-client-skaha.session - ERROR - 404 Client Error: Not Found for url: https://ws-uv.canfar.net/skaha/v0/session/om2svvrz?view=event
2024-03-18 00:06:46,174 ERROR 404 Client Error: Not Found for url: https://ws-uv.canfar.net/skaha/v0/session/om2svvrz?view=event


IndexError: list index out of range

In [217]:
for sid in session_ids:
    
    print(session.logs(sid)[sid])

Execute bash script.
NAME = 587731173842027508_8623-6104
Running CASA pipeline...
--> CrashReporter initialized.
casalogger-x86_64.AppImage: cannot connect to X server 

optional configuration file config.py not found, continuing CASA startup without it
Using user-supplied startup.py at ~/.casa/startup.py

Using matplotlib backend: agg
Creating a new telemetry file
casaVersion =  6.5.6.22
imported casatasks and casatools individually
CASA 6.5.6.22 -- Common Astronomy Software Applications [6.5.6.22]
CASA version:  6.5.6
[INFO] [        build_key_handler] 
[INFO] [        build_key_handler] &%&%&%&%&%&%&%&%&%&%&%&%&%&%&&%&%&%&%&%&%&%&%&%&%
[INFO] [        build_key_handler] Initializing the PHANGS-ALMA pipeline KeyHandler.
[INFO] [        build_key_handler] &%&%&%&%&%&%&%&%&%&%&%&%&%&%&&%&%&%&%&%&%&%&%&%&%
[INFO] [        build_key_handler] 
[INFO] [         _read_master_key] Reading the master key.
[INFO] [         _read_master_key] Master key file: /arc/projects/salvage/phangs_imaging

## Run PHANGS moments

In [55]:
# select appropriate resources

image = "images.canfar.net/skaha/astroml:latest"
#image = "images.canfar.net/casa-5/casa:5.6.1-8-pipeline"
cmd = '/arc/projects/salvage/ALMA_reduction/bash_scripts/run_PHANGS_moments.sh'
ram=4
cores=2

session_ids = []

# testing targets that have empty mom0 maps
#rerun_targets = ['587726033309335775', '588010359075700997', '587727944034091239']
rerun_targets = ['587727944034091201','587727944034091092']

# loop over galaxies and launch jobs
for i in range(len(objID_sample)):

    ID = objID_sample[i]
    NAME = name_sample[i]

    # running only on problem galaxies
    #if ID not in rerun_targets:
    #    continue

    # launch headless session on the science platform
    session = Session()
    session_id = session.create(
        name  = ID,
        image = image,
        cores = cores,
        ram   = ram,
        kind  = "headless",
        cmd   = cmd,
        args  = f'{NAME} {ID}'
    )

    print("Sesion ID: {}".format(session_id[0]))
    session_ids.append(session_id[0])


2024-03-19 14:43:12,088 - skaha-client-skaha.session - INFO - Creating 1 session(s) with parameters:
2024-03-19 14:43:12,088 INFO Creating 1 session(s) with parameters:
2024-03-19 14:43:12,090 - skaha-client-skaha.session - INFO - {'name': '587724198283247682', 'image': 'images.canfar.net/skaha/astroml:latest', 'cores': 2, 'ram': 4, 'kind': 'headless', 'cmd': '/arc/projects/salvage/ALMA_reduction/bash_scripts/run_PHANGS_moments.sh', 'args': 'CP_9195_9101 587724198283247682', 'env': {}}
2024-03-19 14:43:12,090 INFO {'name': '587724198283247682', 'image': 'images.canfar.net/skaha/astroml:latest', 'cores': 2, 'ram': 4, 'kind': 'headless', 'cmd': '/arc/projects/salvage/ALMA_reduction/bash_scripts/run_PHANGS_moments.sh', 'args': 'CP_9195_9101 587724198283247682', 'env': {}}
2024-03-19 14:43:13,177 - skaha-client-skaha.session - INFO - Creating 1 session(s) with parameters:
2024-03-19 14:43:13,177 INFO Creating 1 session(s) with parameters:
2024-03-19 14:43:13,179 - skaha-client-skaha.sessio

Sesion ID: uwx7i8es


2024-03-19 14:43:14,608 - skaha-client-skaha.session - INFO - Creating 1 session(s) with parameters:
2024-03-19 14:43:14,608 INFO Creating 1 session(s) with parameters:
2024-03-19 14:43:14,611 - skaha-client-skaha.session - INFO - {'name': '587730847423791750', 'image': 'images.canfar.net/skaha/astroml:latest', 'cores': 2, 'ram': 4, 'kind': 'headless', 'cmd': '/arc/projects/salvage/ALMA_reduction/bash_scripts/run_PHANGS_moments.sh', 'args': 'PM1_8623_1902 587730847423791750', 'env': {}}
2024-03-19 14:43:14,611 INFO {'name': '587730847423791750', 'image': 'images.canfar.net/skaha/astroml:latest', 'cores': 2, 'ram': 4, 'kind': 'headless', 'cmd': '/arc/projects/salvage/ALMA_reduction/bash_scripts/run_PHANGS_moments.sh', 'args': 'PM1_8623_1902 587730847423791750', 'env': {}}


Sesion ID: hp89bw85


2024-03-19 14:43:16,155 - skaha-client-skaha.session - INFO - Creating 1 session(s) with parameters:
2024-03-19 14:43:16,155 INFO Creating 1 session(s) with parameters:
2024-03-19 14:43:16,157 - skaha-client-skaha.session - INFO - {'name': '587731173842027508', 'image': 'images.canfar.net/skaha/astroml:latest', 'cores': 2, 'ram': 4, 'kind': 'headless', 'cmd': '/arc/projects/salvage/ALMA_reduction/bash_scripts/run_PHANGS_moments.sh', 'args': '587731173842027508_8623-6104 587731173842027508', 'env': {}}
2024-03-19 14:43:16,157 INFO {'name': '587731173842027508', 'image': 'images.canfar.net/skaha/astroml:latest', 'cores': 2, 'ram': 4, 'kind': 'headless', 'cmd': '/arc/projects/salvage/ALMA_reduction/bash_scripts/run_PHANGS_moments.sh', 'args': '587731173842027508_8623-6104 587731173842027508', 'env': {}}


...

Sesion ID: l4kuek7y


2024-03-19 14:44:51,205 - skaha-client-skaha.session - INFO - Creating 1 session(s) with parameters:
2024-03-19 14:44:51,205 INFO Creating 1 session(s) with parameters:
2024-03-19 14:44:51,208 - skaha-client-skaha.session - INFO - {'name': '587724234256351256', 'image': 'images.canfar.net/skaha/astroml:latest', 'cores': 2, 'ram': 4, 'kind': 'headless', 'cmd': '/arc/projects/salvage/ALMA_reduction/bash_scripts/run_PHANGS_moments.sh', 'args': 'PM1_9195_3703 587724234256351256', 'env': {}}
2024-03-19 14:44:51,208 INFO {'name': '587724234256351256', 'image': 'images.canfar.net/skaha/astroml:latest', 'cores': 2, 'ram': 4, 'kind': 'headless', 'cmd': '/arc/projects/salvage/ALMA_reduction/bash_scripts/run_PHANGS_moments.sh', 'args': 'PM1_9195_3703 587724234256351256', 'env': {}}


Sesion ID: a509kb8c


2024-03-19 14:44:52,397 - skaha-client-skaha.session - INFO - Creating 1 session(s) with parameters:
2024-03-19 14:44:52,397 INFO Creating 1 session(s) with parameters:
2024-03-19 14:44:52,410 - skaha-client-skaha.session - INFO - {'name': '587739719757922345', 'image': 'images.canfar.net/skaha/astroml:latest', 'cores': 2, 'ram': 4, 'kind': 'headless', 'cmd': '/arc/projects/salvage/ALMA_reduction/bash_scripts/run_PHANGS_moments.sh', 'args': '8952-6104 587739719757922345', 'env': {}}
2024-03-19 14:44:52,410 INFO {'name': '587739719757922345', 'image': 'images.canfar.net/skaha/astroml:latest', 'cores': 2, 'ram': 4, 'kind': 'headless', 'cmd': '/arc/projects/salvage/ALMA_reduction/bash_scripts/run_PHANGS_moments.sh', 'args': '8952-6104 587739719757922345', 'env': {}}


Sesion ID: du5hqnrx
Sesion ID: kb62ynie


In [56]:
for i,sid in enumerate(session_ids):

    info = session.info(sid)[0]
    print(info['id'], info['status'])

uwx7i8es Succeeded
hp89bw85 Succeeded
yu4j7zm3 Succeeded
pdek7ut3 Succeeded
tahvwoik Succeeded
irwn4rp8 Succeeded
evns9fws Succeeded
qr0vy0t0 Succeeded
kjxm00t1 Succeeded
m33mmmho Succeeded
vifo3kel Succeeded
pic4pecf Succeeded
r2i4vbi0 Succeeded
a1te72it Succeeded
r2zptatr Succeeded
rn9loy9c Succeeded
ryhltg49 Succeeded
ebhvkzy9 Succeeded
dgum59pn Succeeded
ltozhkvf Succeeded
epar84z8 Succeeded
xkqr0tm1 Succeeded
ia2qisnu Succeeded
j0w8vhsm Succeeded
q6l4irz3 Succeeded
e020o44d Succeeded
ey0dgez4 Succeeded
o9uj54pl Succeeded
uaz0zodo Succeeded
mdju6agp Succeeded
ktytmgxj Succeeded
xvi702jf Succeeded
vanrwirs Succeeded
oa68n75n Succeeded
ar49a53j Succeeded
c544104c Succeeded
tiwvt1px Succeeded
k7a7l1w5 Succeeded
uxpniw2l Succeeded
t5o41mnj Succeeded
e7i7p7y8 Succeeded
fr2019lp Succeeded
cv4n3ep5 Succeeded
f7qie5gy Succeeded
epihzmsi Succeeded
qr1hj8kh Succeeded
gsiqg09j Succeeded
aoz7m2yu Succeeded
ucq96lym Succeeded
lq49xpub Succeeded
yu5oux7p Succeeded
w18q8hzs Succeeded
udhqgpqx Suc

In [152]:
for sid in session_ids:
    
    print(session.logs(sid)[sid])

Execute bash script.
NAME = 587731173842027508_8623-6104
OBJID = 587731173842027508
Downgrading astropy
Defaulting to user installation because normal site-packages is not writeable
  Using cached astropy-5.3-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl.metadata (9.5 kB)
Using cached astropy-5.3-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (10.1 MB)
ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: '/arc/home/swilkinson/.local/lib/python3.10/site-packages/astropy/io/votable/data/ucd1p-words.txt'


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Running derived pipeline...
[DEBU] [                 <module>] casa_enabled = False
[INFO] [        build_key_handler] 
[INFO] [        build_key_handler] &%&%&%&%&%&%&%&%&%&%&%&%&%&%&&%&%&%&%&%&%&%&%&%&%
[INFO] [        build_key_handler] I